In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()
df = pd.read_csv('~/data/dataset03.csv')
df=df.drop(['DATETIME'],axis=1)
df_label=df.iloc[:,-1]
df=df.iloc[:,:-1]
df_colms=df.columns
scaler=StandardScaler()
df=scaler.fit_transform(df)
df=pd.DataFrame(df,columns=df_colms)
df=pd.concat((df,df_label),axis=1)



In [3]:
df

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,-1.801668,-0.842161,-1.489735,-1.437100,-0.125305,-0.302092,-2.099840,-0.191837,0.0,0.699708,...,0.968475,-0.470930,0.195802,-1.257323,0.754447,-1.255191,-0.149604,-1.032124,-0.363355,0
1,-1.882432,-0.869352,-0.824341,-1.364756,0.514449,0.701151,-1.750272,-0.182138,0.0,0.702016,...,1.186235,-0.473974,0.421522,-1.269177,0.771176,-1.266938,-0.139625,-1.039972,-0.375851,0
2,-1.959306,-0.885000,-0.090841,-0.577060,1.124164,0.730143,-1.274787,-0.249309,0.0,0.686032,...,1.376247,-0.473960,0.512476,-1.099823,0.955458,-1.096348,0.846406,-1.039905,-0.375745,0
3,-1.948692,-0.869262,0.695223,0.326835,1.638064,0.730143,-0.119569,-0.333484,0.0,0.666001,...,1.558041,-0.441443,0.704155,0.356598,-0.981428,0.351616,1.569917,-1.017362,-0.339848,0
4,-1.823477,-0.814892,1.527235,1.559097,1.029043,0.730143,1.371911,-0.355788,0.0,0.660693,...,1.780077,-0.387166,1.024304,0.397696,-0.973794,0.392623,1.650446,-0.987991,-0.293079,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,0.559571,0.073394,-0.733239,-0.462470,-1.049197,0.730143,1.803126,-0.431049,0.0,0.642784,...,-0.967527,0.427840,0.242993,-0.764829,0.697331,-0.764188,0.653599,-0.525215,0.443826,0
8757,0.486731,0.308065,-1.300971,-0.751625,-0.534049,-0.917044,0.230214,-0.445875,0.0,0.639256,...,-1.188568,0.540887,0.258070,0.576591,0.883010,0.594349,-2.952231,-0.453379,0.558214,0
8758,0.363945,0.451059,-1.589951,-0.940375,-0.210137,-2.101309,-1.321260,-0.397785,0.0,0.650699,...,0.876053,0.213828,0.247399,-0.879113,0.723928,-0.879906,-0.267414,-0.584710,0.349089,0
8759,0.211740,0.413257,-0.971169,-1.005800,0.238745,-1.497369,-1.485046,-0.370177,0.0,0.657269,...,0.993773,0.191520,0.166530,-0.906476,0.756239,-0.906923,-0.202511,-0.601332,0.322620,0


In [4]:

df_normal=df[df['ATT_FLAG']==0]
df_attack=df[df['ATT_FLAG']==1]


In [5]:
df_normal

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,-1.801668,-0.842161,-1.489735,-1.437100,-0.125305,-0.302092,-2.099840,-0.191837,0.0,0.699708,...,0.968475,-0.470930,0.195802,-1.257323,0.754447,-1.255191,-0.149604,-1.032124,-0.363355,0
1,-1.882432,-0.869352,-0.824341,-1.364756,0.514449,0.701151,-1.750272,-0.182138,0.0,0.702016,...,1.186235,-0.473974,0.421522,-1.269177,0.771176,-1.266938,-0.139625,-1.039972,-0.375851,0
2,-1.959306,-0.885000,-0.090841,-0.577060,1.124164,0.730143,-1.274787,-0.249309,0.0,0.686032,...,1.376247,-0.473960,0.512476,-1.099823,0.955458,-1.096348,0.846406,-1.039905,-0.375745,0
3,-1.948692,-0.869262,0.695223,0.326835,1.638064,0.730143,-0.119569,-0.333484,0.0,0.666001,...,1.558041,-0.441443,0.704155,0.356598,-0.981428,0.351616,1.569917,-1.017362,-0.339848,0
4,-1.823477,-0.814892,1.527235,1.559097,1.029043,0.730143,1.371911,-0.355788,0.0,0.660693,...,1.780077,-0.387166,1.024304,0.397696,-0.973794,0.392623,1.650446,-0.987991,-0.293079,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,0.559571,0.073394,-0.733239,-0.462470,-1.049197,0.730143,1.803126,-0.431049,0.0,0.642784,...,-0.967527,0.427840,0.242993,-0.764829,0.697331,-0.764188,0.653599,-0.525215,0.443826,0
8757,0.486731,0.308065,-1.300971,-0.751625,-0.534049,-0.917044,0.230214,-0.445875,0.0,0.639256,...,-1.188568,0.540887,0.258070,0.576591,0.883010,0.594349,-2.952231,-0.453379,0.558214,0
8758,0.363945,0.451059,-1.589951,-0.940375,-0.210137,-2.101309,-1.321260,-0.397785,0.0,0.650699,...,0.876053,0.213828,0.247399,-0.879113,0.723928,-0.879906,-0.267414,-0.584710,0.349089,0
8759,0.211740,0.413257,-0.971169,-1.005800,0.238745,-1.497369,-1.485046,-0.370177,0.0,0.657269,...,0.993773,0.191520,0.166530,-0.906476,0.756239,-0.906923,-0.202511,-0.601332,0.322620,0


In [6]:
df_test=pd.read_csv('~/data/test_dataset.csv')
df_test=df_test.drop(['DATETIME'],axis=1)
df_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,98.95,...,90.54,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0
1,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,97.97,...,90.43,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0
2,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,96.84,...,91.72,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0
3,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,96.78,...,76.12,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0
4,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,94.79,...,75.68,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,70.07,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0
2085,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,68.29,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0
2086,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,87.08,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0
2087,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,87.85,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0


In [7]:
y_actual=df_test.iloc[:,-1]
df_test=df_test.iloc[:,:-1]
scaler=StandardScaler()
df_test=scaler.fit_transform(df_test)
df_test=pd.DataFrame(df_test,columns=df_colms)
df_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,-1.691904,-0.752059,-0.293836,-0.571260,1.516235,0.726698,1.204044,-0.109271,0.069354,0.735521,...,-0.449652,1.281197,-0.435599,0.215417,-1.142721,0.890069,-1.150464,0.648516,-0.994371,-0.346750
1,-1.724926,-0.765939,0.489027,-0.571260,1.475512,0.726698,1.815205,-0.189570,0.069354,0.712890,...,-0.453103,1.268529,-0.439075,0.283150,0.318342,-1.016702,0.312475,1.109080,-0.989524,-0.339299
2,-1.551562,-0.724300,1.227578,-0.295172,0.769642,-1.247388,-0.091619,-0.282158,0.069354,0.686794,...,-0.397882,1.417087,-0.383468,0.559292,1.035468,-1.057740,1.056266,-1.650556,-0.948316,-0.275972
3,-1.378199,-0.564682,1.419602,0.735555,0.090921,-2.118307,-0.922798,-0.287075,0.069354,0.685409,...,0.099115,-0.379426,0.082237,0.776818,0.443448,-1.096776,0.435693,0.571756,-0.705916,0.096541
4,-1.246112,-0.252387,1.094639,1.287731,-0.547077,0.029962,0.140623,-0.450129,0.069354,0.639454,...,1.151781,-0.430097,1.156139,-1.734508,0.651214,-1.095775,0.644044,0.564267,-0.024775,1.143304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,-0.693000,-0.876977,-0.781280,-1.141841,-1.375117,-2.408614,-1.472844,1.617139,0.069354,-1.549541,...,-0.273633,-1.076151,-0.293107,0.154197,-1.399635,0.601801,-1.410343,-0.354987,-0.982252,-0.328124
2085,-1.023216,-0.752059,-1.416434,-1.344305,-0.927161,-2.060246,-1.778424,1.799859,0.069354,-1.549541,...,-0.297792,-1.281138,-0.317435,0.033060,-1.676656,0.546750,-1.688144,-0.323160,-1.013763,-0.376551
2086,-1.411220,-0.772879,-1.785709,-2.338221,-0.560652,-1.479633,-1.851764,1.817065,0.069354,-1.549541,...,-0.639477,0.882740,-0.623271,0.117726,-1.692294,0.587788,-1.703827,-0.295077,-1.147083,-0.581433
2087,-1.592839,-0.842278,-1.121014,-2.246192,-0.112696,-0.782897,-1.802871,-0.127298,0.069354,0.730440,...,-0.501423,0.971414,-0.487730,0.156802,-1.238784,0.699892,-1.249039,-0.336265,-1.023459,-0.391451


In [8]:
labels=hankel.fit(np.array(y_actual),1,0.5)
isatt=np.any(labels>0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([ 8,  9, 10, 11, 12, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 39, 40,
       41, 42, 43, 44, 51, 52, 53, 54, 55, 63, 64, 65])

In [9]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    # print(indices[idx])
    return radiis[indices[idx]]

In [10]:
len(df_normal.columns)

44

In [11]:
for sens in range(44):    
    X = df_normal.iloc[:,sens].values
    X = hankel.fit(X,1,0.5)
    X=X.T 
    kmeans,optimal_k = cluster.fit(X)
    # optimal_k = 16
    kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
    kmeans.fit(X)
    print(optimal_k)
    radiis_normal = []
    weights=[]
    centers=[]
    clusters_R=[]
    clusters_V=[]
    for i in range(optimal_k):
        cluster_ = X[np.where(kmeans.labels_ == i)[0]]
        r = rank.fit(cluster_)
        clusters_R.append(r)
        # print("r: "+str(r))
        U,Sigma,VT = np.linalg.svd(cluster_)
        V = VT.T
        clusters_V.append(V)
        cluster_ = np.matmul(cluster_,V[:,:r])
        weight,center = meepc.fit(cluster_)
        weights.append(weight)
        centers.append(center)
        var1=np.square(cluster_-center)
        var2=np.matmul(weight,var1.T)
        radiis_normal.append(np.amax(np.sqrt(var2)))
    
    X_test=df_test.iloc[:,sens].values
    Xtest=hankel.fit(X_test,1,0.5)  #size -> 1800x274
    Xtest=Xtest.T  
    distances=[]
    for i in range(optimal_k):
        Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) 
        var1=np.square(Xtest_cluster-centers[i])
        var2=np.matmul(weights[i],var1.T)
        distance=np.sqrt(var2)
        distances.append(distance)

    stacked_arr=np.vstack(distances)
    distances_arr=np.transpose(stacked_arr)
    check_anamoly=radiis_normal-distances_arr     # positive = inside cluster | negative = outside cluster
    #check for if any datapoint is coming inside the cluster
    is_positive=np.any(check_anamoly>0,axis=1)
    #indices of those rows
    positive_indices=np.where(is_positive)[0]
    # for i in [[267, 268, 269, 272, 273]]:
    #     print(distances_arr[i])
    for i in range(len(Xtest)):
        if i not in positive_indices:
            print(i,end=' ')
    print("\nnext")



5
0 20 23 28 30 31 32 34 40 41 43 62 63 
next
5
9 10 20 21 28 46 60 62 
next
2
20 21 22 47 
next
2
30 37 64 67 
next
2

next
5
34 50 
next
5
7 11 15 16 21 25 43 66 
next
5
30 31 
next


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
X_test=df_test.iloc[:,].values
Xtest=hankel.fit(X_test,1,0.5)  #size -> 1800x274
Xtest=Xtest.T  
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) 
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
check_anamoly=radiis_normal-distances_arr     # positive = inside cluster | negative = outside cluster
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]
# for i in [[267, 268, 269, 272, 273]]:
#     print(distances_arr[i])
for i in range(len(Xtest)):
    if i not in positive_indices:
        print(i,end=' ')
print("\nnext")

In [ ]:
array([ 8,  9, 10, 11, 12, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 39, 40,
       41, 42, 43, 44, 51, 52, 53, 54, 55, 63, 64, 65])

In [ ]:
16
0 11 12 13 20 21 22 23 28 29 30 31 36 37 40 41 42 43 45 46 54 55 58 61 62 63 67 
next
16
0 1 4 8 9 10 11 12 18 20 21 22 27 28 30 36 39 40 41 42 43 44 45 49 51 55 56 58 60 61 62 64 65 66 67 
next
16
6 8 9 10 11 12 13 14 16 17 18 19 20 21 22 24 29 31 34 35 42 43 50 59 61 
next
16
31 35 36 37 66 
next
16
3 5 6 8 9 12 13 17 20 21 22 24 27 31 35 37 38 49 56 61 62 65 66 
next
16
8 31 66 
next
16
22 41 
next
16
11 18 20 27 28 30 31 33 35 42 44 56 58 62 63 66 
next
16